In [1]:
#| default_exp core

In [2]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

In [3]:
#| export
from collections import Counter
from itertools import cycle

import numpy as np
from fastcore.basics import patch

from matatena.exceptions import ColumnFullError

# Basic rules

Knucklebones is a simple game in which a dice is rolled and one of the two players must place it in their corresponding board. Each player can only choose the column in which the dice is placed, and if any dice is already placed in that column, the following is put behind. If there are more than one dice with the same number, their score is multiplied. For example, if a column has two 6s, their values add up to 36, but if a column has a 6 and a 5, the total score would be 11.

> The basic board is a 3x3 grid.

# Definition of the board

Each game board (one for each player) can be represented, for example, with a `np.array`. But we may probably benefit from having a "global" object that controls both of the players to keep track of both boards and the score in a more simple manner.

In [25]:
#| export

class Game():
    """
    Class that controls the whole game. It keeps track of both boards.
    """
    def __init__(self,
                 n_players=2, # Number of players.
                 board_size=3, # Size of the board. It is a squared grid of `board_size`x`board_size`
                 ):
        self.n_players = n_players
        self.board_size = board_size
        self.boards = np.zeros(shape=(n_players, board_size, board_size))
        self.current_player = self.choose_initial_player()
        self._players = cycle(range(n_players))

    def __repr__(self):
        """
        Representation of the game state.
        The current player is marked with an *.
        """
        player0 = "Player 1 *" if self.current_player==0 else "Player 1"
        player1= "Player 2 *" if self.current_player==1 else "Player 2"

        return "\n".join([player0,
                          str(self.boards[0]),
                          player1,
                          str(self.boards[1])])
        
    def choose_initial_player(self):
        """
        The initial player is chosen at random at the beggining of the game.
        """
        return np.random.choice([0,1])
    
    def _change_player(self):
        """
        Changes the current player.
        """
        return next(self._players)

    def is_done(self):
        """
        The game is considered finished when one of the boards is completed.
        This can be checked by checking if there are any 0s left in a board.
        """
        return (not (self.boards[0] == 0).any()) or (not (self.boards[1] == 0).any())

    def add_dice(self,
                 player, # Player to add the dice to.
                 column, # Column where we want to place the dice.
                 dice, # Dice to place.
                 ):
        """
        Adds a dice to the corresponding player in the specified column.
        """
        board_column = self.boards[player][:,column]
        idxs_dice = np.where(board_column==0)[0]
        if len(idxs_dice)==0: raise ColumnFullError # Can't place dice in a full column.
        board_column[idxs_dice[0]] = dice
        self.current_player = self._change_player()
        return self.is_done()

In [5]:
matatena = Game()
assert not matatena.is_done()
matatena

Player 1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

In [6]:
matatena = Game()
matatena.boards[1] = np.ones_like(matatena.boards[1])
assert matatena.is_done()
matatena

Player 1 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

In [26]:
matatena = Game()
matatena.add_dice(player=0, column=0, dice=6)
print(matatena)
matatena.add_dice(player=0, column=0, dice=6)
print(matatena)

Player 1 *
[[6. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 2
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Player 1
[[6. 0. 0.]
 [6. 0. 0.]
 [0. 0. 0.]]
Player 2 *
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## Calculating the score

In [8]:
#| export

@patch
def score(self: Game,
          player, # Number of the player we want to calculate the score.
          ):
    """
    Returns the calculated score for a player. 
    If there are numbers repeated in a column, 
    their values must be added and multiplied by the number of repetitions. 
    Otherwise, they are added. If there are repreated and non-repeated in the same column, 
    the repeated are summed and multiplied by the number of repetitions and then the result is added to the non-repeated.
    """
    total = 0
    for col in self.boards[player].T: # Transposed to iterate over the columns
        cntr = Counter(col)
        sum_col = sum([n*reps*reps for n, reps in cntr.items()])
        total += sum_col
    return total

In [9]:
matatena = Game()
matatena.boards[0] = np.array([[1,0,0],
                               [1,2,3],
                               [4,2,5]])
assert matatena.score(0) == 8+8+8